# Spherical ICET

In [2]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp

physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)
    
%load_ext autoreload
%autoreload 2
%autosave 180

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [250]:
from ICET_spherical import ICET

## init KITTI dataset -----------------------------------------------------------------
basedir = 'C:/kitti/'
date = '2011_09_26'
drive = '0005'
idx = 10
frame_range = range(150, 151, 1)
dataset = pykitti.raw(basedir, date, drive)
velo1 = dataset.get_velo(idx) # Each scan is a Nx4 array of [x,y,z,reflectance]
c1 = velo1[:,:3]
c1 = c1[c1[:,2] > -1.5] #ignore ground plane
velo2 = dataset.get_velo(idx+1) # Each scan is a Nx4 array of [x,y,z,reflectance]
c2 = velo2[:,:3]
c2 = c2[c2[:,2] > -1.5] #ignore ground plane
## ------------------------------------------------------------------------------------

#single distinct cluster
# c1 = np.random.randn(3000,3)*tf.constant([0.3,0.04,0.3]) + tf.constant([0.,4.,0.])
# c2 = np.random.randn(3000,3)*tf.constant([0.3,0.04,0.3]) + tf.constant([0.,4.,0.]) - np.array([0.1, 0.3, 0.0])
# c2 = c1 - np.array([0.1, 0.3, 0.0])

# D = True
D = False
X = tf.constant([0.3, 0., 0., 0., 0., 0.])
it = ICET(cloud1 = c1, cloud2 = c2,  fid = 50, draw = D, x0 = X)
ViewInteractiveWidget(it.plt.window)

estimated solution vector X: 
 tf.Tensor([0.3 0.  0.  0.  0.  0. ], shape=(6,), dtype=float32)
estimated solution vector X: 
 tf.Tensor(
[ 2.9021862e-01  1.5780320e-02  4.5703095e-03  1.1463766e-05
 -5.8212602e-03  1.1957903e-02], shape=(6,), dtype=float32)
estimated solution vector X: 
 tf.Tensor([ 0.29367393  0.00832375  0.00971818 -0.0005553  -0.00776695  0.01230285], shape=(6,), dtype=float32)
estimated solution vector X: 
 tf.Tensor([ 0.29397506  0.00797868  0.01166027 -0.00081911 -0.00800717  0.01232892], shape=(6,), dtype=float32)
estimated solution vector X: 
 tf.Tensor([ 0.29395065  0.00795303  0.01179247 -0.00084146 -0.00800735  0.01233483], shape=(6,), dtype=float32)
stds: 
 tf.Tensor(
[7.8877457e-04 9.3999226e-04 1.9912678e-03 2.8049873e-04 2.6349569e-04
 7.3088413e-05], shape=(6,), dtype=float32)


AttributeError: 'ICET' object has no attribute 'plt'

In [242]:
#get true transformation between frames
from metpy.calc import lat_lon_grid_deltas
poses0 = dataset.oxts[idx] #<- ID of 1st scan
poses1 = dataset.oxts[idx+1] #<- ID of 2nd scan
lat0 = poses0.packet.lat
lon0 = poses0.packet.lon
alt0 = poses0.packet.alt
lat1 = poses1.packet.lat
lon1 = poses1.packet.lon
alt1 = poses1.packet.alt

dx_oxts, dy_oxts = lat_lon_grid_deltas(np.array([lon0,lon1]), np.array([lat0, lat1]))
# print(dx_oxts, dy_oxts) 
dx_oxts = dx_oxts[0,0].magnitude
dy_oxts = dy_oxts[0,0].magnitude
dz_oxts = (alt0-alt1)
droll_oxts = (poses0.packet.roll - poses1.packet.roll)
dpitch_oxts = (poses0.packet.pitch - poses1.packet.pitch)
dyaw_oxts = (poses0.packet.yaw - poses1.packet.yaw)

rot = poses1.T_w_imu[:3,:3] #trying this

dxyz_oxts = np.array([[dx_oxts, dy_oxts, dz_oxts]])
dxyz_lidar = dxyz_oxts.dot(rot)
print(dxyz_lidar)

dt = 0.10
from_vel = np.array([[poses1.packet.vf*dt, poses1.packet.vl*dt, poses1.packet.vu*dt, -poses1.packet.wf*dt, -poses1.packet.wl*dt, -poses1.packet.wu*dt]])
print(from_vel)


[[ 0.30123163 -0.02423234  0.00349136]]
[[ 0.31619813  0.00952271 -0.01318738 -0.00226272  0.00075722 -0.01186753]]


In [ ]:
#test where points are inside spherical cell...
# print(it.cloud1_tensor_spherical)
maxtheta = tf.constant([[0.2],[0.7]])
maxr = tf.constant([[0.5],[2.]])

ans1 = tf.greater(it.cloud1_tensor_spherical[:,1], maxtheta)
# print(ans1)
ans2 = tf.less(it.cloud1_tensor_spherical[:,0], maxr)
# print(ans2)
combined = tf.Variable([ans1, ans2])
# print(combined)
ans3 = tf.math.reduce_all(combined, axis = 1)

print(ans3)

In [42]:
#duplicate each element of an n*1 vector 3 times
t = tf.linspace(0,5,6)[:,None]
print(t)

test  = tf.tile(t, [3,1])
# print(test)
test2 = tf.reshape(tf.transpose(tf.reshape(test, [3,-1])), [-1,1])
print(test2)
test3 = tf.reshape(tf.transpose(tf.reshape(test, [3,-1])), [-1,3])
print(test3)

tf.Tensor(
[[0.]
 [1.]
 [2.]
 [3.]
 [4.]
 [5.]], shape=(6, 1), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [2.]
 [2.]
 [2.]
 [3.]
 [3.]
 [3.]
 [4.]
 [4.]
 [4.]
 [5.]
 [5.]
 [5.]], shape=(18, 1), dtype=float64)
tf.Tensor(
[[0. 0. 0.]
 [1. 1. 1.]
 [2. 2. 2.]
 [3. 3. 3.]
 [4. 4. 4.]
 [5. 5. 5.]], shape=(6, 3), dtype=float64)
